In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import os
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, List, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData
from gpu_spatial_graph_pipeline.utils import adata2data, to_one_data
from gpu_spatial_graph_pipeline.data.datamodule import GraphAnnDataModule
from gpu_spatial_graph_pipeline.models.linear_ncem import LinearNCEM

In [34]:
#Mibitof
adata = sq.datasets.mibitof()
#feature_name=adata.obs.keys()[0] #Use for IMC dataset

#specify features to use
feature_names=['Cluster','batch']

#input of datamodule
num_features=(len(set(adata.obs[feature_names[0]])),len(set(adata.obs[feature_names[1]])))
num_genes=adata.X.shape[1]

def mibitof2data(adata):
    return [adata2data(adata,feature_names)[0]]


In [42]:
dm = GraphAnnDataModule(adata=adata, adata2data_fn=mibitof2data, num_workers = 8, batch_size=40,learning_type='nodewise')

In [43]:
dm.setup()

In [44]:
for batch in dm.train_dataloader():
    print(batch)

Data(x=[500, 11], edge_index=[2, 1452], y=[500, 36], train_mask=[500], val_mask=[500], test_mask=[500], batch_size=40)
Data(x=[551, 11], edge_index=[2, 1572], y=[551, 36], train_mask=[551], val_mask=[551], test_mask=[551], batch_size=40)
Data(x=[513, 11], edge_index=[2, 1518], y=[513, 36], train_mask=[513], val_mask=[513], test_mask=[513], batch_size=40)
Data(x=[515, 11], edge_index=[2, 1536], y=[515, 36], train_mask=[515], val_mask=[515], test_mask=[515], batch_size=40)
Data(x=[530, 11], edge_index=[2, 1554], y=[530, 36], train_mask=[530], val_mask=[530], test_mask=[530], batch_size=40)
Data(x=[522, 11], edge_index=[2, 1512], y=[522, 36], train_mask=[522], val_mask=[522], test_mask=[522], batch_size=40)
Data(x=[476, 11], edge_index=[2, 1434], y=[476, 36], train_mask=[476], val_mask=[476], test_mask=[476], batch_size=40)
Data(x=[488, 11], edge_index=[2, 1434], y=[488, 36], train_mask=[488], val_mask=[488], test_mask=[488], batch_size=40)
Data(x=[516, 11], edge_index=[2, 1506], y=[516, 

In [45]:
model = LinearNCEM(in_channels=num_features,out_channels=num_genes, model_type='spatial', lr=0.0001,weight_decay=0.000001)

In [46]:
gpu=False
if gpu:
    trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=100,log_every_n_steps=1)
else:
    trainer:pl.Trainer = pl.Trainer(accelerator='cpu',max_epochs=100,log_every_n_steps=1)


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/sel/anaconda3/envs/graph/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [47]:
trainer.fit(model,datamodule=dm)


  | Name        | Type            | Params
------------------------------------------------
0 | model_sigma | LinearSpatial   | 2.7 K 
1 | model_mu    | LinearSpatial   | 2.7 K 
2 | loss_module | GaussianNLLLoss | 0     
------------------------------------------------
5.5 K     Trainable params
0         Non-trainable params
5.5 K     Total params
0.022     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/sel/anaconda3/envs/graph/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/sel/anaconda3/envs/graph/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/sel/anaconda3/envs/graph/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` 

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/sel/anaconda3/envs/graph/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
